In [11]:
import matplotlib.pyplot as plt, rdflib, pandas as pd, numpy as np, sys, os, random, math, fiona, uuid, copy, glob
from osgeo import gdal, osr, gdal_array
from collections import defaultdict, Counter
from dotenv import load_dotenv
from tqdm.auto import tqdm
from typing import *
from ruamel.yaml import YAML
import xarray as xr

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# next cell
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
load_dotenv(verbose=True)
paths = ["../", "/workspace/d-repr/pydrepr", "/home/rook/workspace/d-repr/pydrepr"]
for path in paths:
    if path not in sys.path:
        sys.path.insert(0, path)

yaml = YAML()

from drepr import __version__, DRepr, outputs
from drepr.executors.readers.reader_container import ReaderContainer
from drepr.executors.readers.np_dict import NPDictReader
print("drepr version:", __version__)

drepr version: 2.8


In [3]:
from funcs import DcatReadFunc
from funcs.readers.dcat_read_func import ShardedClassID
from funcs.gdal.raster import *
from dateutil.parser import parse

**configuration & global variables**

In [4]:
HOME_DIR = os.environ['HOME_DIR']

gldas = "5babae3f-c468-4e01-862e-8b201468e3b5"
gpm = "ea0e86f3-9470-4e7e-a581-df85b4a7075d"
variable = "atmosphere_water__precipitation_mass_flux"
variable = "land_surface_air__temperature"

ethiopia = BoundingBox(32.75418, 3.22206, 47.98942, 15.15943)

**download the weather dataset**

In [12]:
def read_datasets(dataset_id, start_time, end_time):
  func = DcatReadFunc(dataset_id, parse(start_time), parse(end_time))
  func.set_preferences({"data": "array"})
  datasets = func.exec()['data']
  return datasets

def read_local_datasets(repr_file, resource_path):
  drepr = DRepr.parse_from_file(repr_file)
  datasets = []
  for file in glob.glob(resource_path):
    datasets.append(outputs.ArrayBackend.from_drepr(drepr, file))  
  if len(datasets) > 1:
    return ShardedBackend(datasets)
  return datasets[0]

In [6]:
weather_dataset = read_datasets(gldas, "2011-09-01T00:00:00", "2011-09-01T03:00:00")

2020-03-04 13:14:02,996 | funcs.readers.dcat_read_func | INFO - Overwrite GLDAS
2020-03-04 13:14:02,997 | funcs.readers.dcat_read_func | INFO - Found key 'resource_repr'
2020-03-04 13:14:02,997 | funcs.readers.dcat_read_func | INFO - Downloading 1 resources ...
2020-03-04 13:14:02,998 | funcs.readers.dcat_read_func | INFO - Download Complete. Skip 1 and download 0 resources


**crop the data**

some useful functions to convert datasets to rasters and convert them back

In [7]:
with open(HOME_DIR + "/examples/d3m/crop_bb.yml", "r") as f:
  crop_bb_conf = yaml.load(f)

def dataset2raster(sm, variable):
  rasters = []
  for c in sm.c('mint:Variable').filter(outputs.FCondition("mint:standardName", "==", variable)):
    for raster_id, sc in c.group_by("mint-geo:raster"):
      # TODO: handle time properly
      timestamp = sc.p("mint:timestamp").as_ndarray([])
      if timestamp.data.size != 1:
        raise NotImplemented()
      timestamp = timestamp.data[0]
      
      data = sc.p("rdf:value").as_ndarray([sc.p("mint-geo:lat"), sc.p("mint-geo:long")])
      gt_info = sm.get_record_by_id(raster_id)
      gt = GeoTransform(x_0=gt_info.s("mint-geo:x_0"),
                        y_0=gt_info.s("mint-geo:y_0"),
                        dx=gt_info.s("mint-geo:dx"), dy=gt_info.s("mint-geo:dy"))
      raster = Raster(data.data, gt, int(gt_info.s("mint-geo:epsg")),
             float(data.nodata.value) if data.nodata is not None else None)
      raster.timestamp = timestamp
      rasters.append(raster)
  return rasters
  
def raster2dataset(r, variable):
  global crop_bb_conf
  reader = NPDictReader({
    "variable": r.data,
    "lat": r.get_center_latitude(),
    "long": r.get_center_longitude(),
    "timestamp": r.timestamp,
    "standard_name": variable,
    "gt_x_0": r.geotransform.x_0,
    "gt_y_0": r.geotransform.y_0,
    "gt_dx": r.geotransform.dx,
    "gt_dy": r.geotransform.dy,
    "gt_epsg": r.epsg,
    "gt_x_slope": r.geotransform.x_slope,
    "gt_y_slope": r.geotransform.y_slope,
  })
  resource_id = str(uuid.uuid4())
  ReaderContainer.get_instance().set(resource_id, reader)
  
  conf = copy.deepcopy(crop_bb_conf)
  conf['attributes']['variable']['missing_values'].append(r.nodata)
  drepr = DRepr.parse(conf)
  sm = outputs.ArrayBackend.from_drepr(drepr, resource_id)
  ReaderContainer.get_instance().delete(resource_id)
  return sm

def raster2netcdf(r, variable, outfile):
  lat = r.get_center_latitude()
  long = r.get_center_longitude()
  data = xr.DataArray(r.data, dims=('lat', 'long'), coords={'lat': lat, 'long': long})
  data.attrs['standard_name'] = variable
  data.attrs['_FillValue'] = r.nodata
  data.attrs['missing_values'] = r.nodata
  
  ds = xr.Dataset({standard_name: data})  
  ds.to_netcdf(outfile)
  
def dataset2netcdf(sm):
  datasets = {}
  for c in sm.c("mint:Variable"):
    if c.p("mint:Place") is not None:
      raise NotImplemented()

    for standard_name, sc1 in c.group_by("mint:standardName"):
      for time, sc2 in sc1.group_by("mint:timestamp"):
        val = sc2.p("rdf:value").as_ndarray([sc2.p("mint-geo:lat"), sc2.p("mint-geo:long")])
        data = val.data.reshape(1, *val.data.shape)
        data = xr.DataArray(val.data.reshape(1, *val.data.shape), dims=('time', 'lat', 'long'), coords={
          'lat': val.index_props[0], 'long': val.index_props[1], 'time': np.asarray([time])
        })
        data.attrs['standard_name'] = standard_name
        data.attrs['_FillValue'] = val.nodata.value
        data.attrs['missing_values'] = val.nodata.value
        
        assert standard_name not in datasets
        datasets[standard_name] = data
  ds = xr.Dataset(datasets)
  ds.attrs.update({
    "conventions": "CF-1.6"
  })
  return ds

crop the data by a bounding box

In [9]:
subrasters = []
for raster in tqdm(dataset2raster(weather_dataset, variable)):
  sr = raster.crop(bounds=ethiopia, resampling_algo=ReSample.BILINEAR)
  sr.timestamp = raster.timestamp
  filename = datetime.datetime.utcfromtimestamp(sr.timestamp).strftime("%Y%m%d%H%M%S")
  sm = raster2dataset(sr, variable)
  dataset2netcdf(sm).to_netcdf(HOME_DIR + f"/data/gldas/{variable}/{filename}.nc4")
  subrasters.append(sr)

debug to see if the data is correct

In [10]:
subrasters[-1].to_geotiff(HOME_DIR + "/data/small.tif")
raster.to_geotiff(HOME_DIR + "/data/full.tif")
sm = read_local_datasets(HOME_DIR + "/examples/d3m/gldas.crop.yml", HOME_DIR + f"/data/gldas/{variable}/*.nc4")
a = dataset2raster(sm, variable)[0].data
b = subrasters[-1].data

assert np.allclose(a, b)

-----------

In [257]:
sm = raster2dataset(rasters[0], variable)

In [272]:
dataset2netcdf(sm).to_netcdf(HOME_DIR + "/data/tmp_out/test.nc4")

In [222]:
for raster in rasters:
  raster.to_geotiff(HOME_DIR + "/data/full.tif")
  ethiopia_raster = raster.crop(bounds=ethiopia, resampling_algo=ReSample.BILINEAR)
  ethiopia_raster.to_geotiff(HOME_DIR + "/data/small.tif")

In [223]:
sm = raster2dataset(ethiopia_raster, variable)

In [235]:
dataset2netcdf(sm)

None
defaultdict(<class 'list'>, {'rdf:value': [0], 'mint-geo:lat': [1], 'mint-geo:long': [2], 'mint:standardName': [3], 'mint-geo:raster': [4], 'mint:timestamp': []})
defaultdict(<class 'list'>, {'rdf:value': [0], 'mint-geo:lat': [1], 'mint-geo:long': [2], 'mint:standardName': [3], 'mint-geo:raster': [4], 'mint:timestamp': []})


IndexError: list index out of range